In [ ]:
# Clone a Git repository
!git clone -b develop https://github.com/teamdatatonic/vertex-pipelines-end-to-end-samples

In [ ]:
%cd vertex-pipelines-end-to-end-samples

In [ ]:
PROJECT_ID = "my-project-id"  # @param {type:"string"}
LOCATION = "europe-west2" # @param {type:"string"}

In [ ]:
# Set the project id
!gcloud config set project {PROJECT_ID}

In [ ]:
!gcloud auth application-default login

In [ ]:
import os

os.environ['PIPELINE_TEMPLATE'] = 'xgboost'
os.environ['VERTEX_CMEK_IDENTIFIER'] = ''
os.environ['VERTEX_LOCATION'] = LOCATION
os.environ['VERTEX_NETWORK'] = ''
os.environ['VERTEX_PROJECT_ID'] = PROJECT_ID
os.environ['RESOURCE_SUFFIX']='default'

# Calculate the value of VERTEX_SA_EMAIL based on VERTEX_PROJECT_ID
os.environ['VERTEX_SA_EMAIL'] = f'vertex-pipelines@{os.environ["VERTEX_PROJECT_ID"]}.iam.gserviceaccount.com'
os.environ['PIPELINE_FILES_GCS_PATH'] = f'gs://{os.environ["VERTEX_PROJECT_ID"]}-pl-root'
os.environ['CONTAINER_IMAGE_REGISTRY']=f'{os.environ["VERTEX_LOCATION"]}-docker.pkg.dev/{os.environ["VERTEX_PROJECT_ID"]}/vertex-images'

In [ ]:
!gcloud services enable cloudresourcemanager.googleapis.com --project={PROJECT_ID}
!gcloud services enable serviceusage.googleapis.com --project={PROJECT_ID}
!gcloud storage buckets create gs://{PROJECT_ID}-tfstate --project={PROJECT_ID} --location={LOCATION} --uniform-bucket-level-access

In [ ]:
!make install

In [ ]:
#Make deploy needs approval (i.e. -auto-approve flag) MAKE FILE WOULD NEED SOME CHANGES
!make deploy

In [ ]:
!bq --location={LOCATION} mk --dataset "{PROJECT_ID}:chicago_taxi_trips"

In [ ]:
!bq --location={LOCATION} mk --dataset "{PROJECT_ID}:preprocessing"

In [ ]:
# TO TRANSFER DATA USED CONSOLE TO CREATE DATA TRANSFER


#ERRORS
# bq mk asks for code to be pasted from generated link to authenticate data transfer. Cant be done in jupyter?

# bq mk --transfer_config \
#   --project_id=${VERTEX_PROJECT_ID} \
#   --data_source="cross_region_copy" \
#   --target_dataset="chicago_taxi_trips" \
#   --display_name="Chicago taxi trip mirror" \
#   --params='{"source_dataset_id":"'"chicago_taxi_trips"'","source_project_id":"'"bigquery-public-data"'"}'


# python code below gives "Request contains an invalid argument. error


#!pip install google-cloud-bigquery-datatransfer
# from google.cloud import bigquery_datatransfer

# transfer_client = bigquery_datatransfer.DataTransferServiceClient()

# destination_project_id = "dt-donatas-sandbox-dev"
# destination_dataset_id = "dt-donatas-sandbox-dev:chicago_taxi_trips"
# source_project_id = "bigquery-public-data"
# source_dataset_id = "chicago_taxi_trips"
# transfer_config = bigquery_datatransfer.TransferConfig(
#     destination_dataset_id=destination_dataset_id,
#     display_name="Chicago taxi trip mirror",
#     data_source_id="cross_region_copy",
#     params={
#         "source_project_id": source_project_id,
#         "source_dataset_id": source_dataset_id,
#     },
#     schedule="every 24 hours",
# )
# transfer_config = transfer_client.create_transfer_config(
#     parent=transfer_client.common_project_path(destination_project_id),
#     transfer_config=transfer_config,
# )
# print(f"Created transfer config: {transfer_config.name}")

In [ ]:
!make build

In [ ]:
!make run pipeline=training build=false

In [ ]:
!make run pipeline=prediction build=false

In [ ]:
#!make undeploy